#### Connect to BigQuery

In [7]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Dinis/ironhack/ironhack_service_account_big_query.json"

In [8]:
import six
six.moves.reload_module(six)

<module 'six' from '/usr/local/lib/python3.7/site-packages/six.py'>

In [9]:
from google.cloud import bigquery

In [10]:
client = bigquery.Client()

## Lab | Advanced MySQL

- In table sales, a title can appear several times. The royalties need to be calculated for each sale.

- Despite a title can have multiple sales records, the advance must be calculated only once for each title.


In your eventual solution, you need to sum up the following profits for each individual author:

- All advances which is calculated exactly once for each title.
- All royalties in each sale.

### Challenge 1: 
    
Step 1: Calculate the royalty of each sale for each author.

Step 2: Using the output from Step 1 as a sub-table, aggregate the total royalties for each title for each author.

Step 3: Using the output from Step 2 as a sub-table, calculate the total profits of each author by aggregating the advances and total royalties of each title.


### Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

- Title ID
- Author ID
- Royalty of each sale for each author

#### The formula is:
sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100

Note that titles.royalty and titleauthor.royaltyper are divided by 100 respectively because they are percentage numbers instead of floats.
In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [64]:
step1 = '''

SELECT
    pti.title_id AS title_id,
    pti.price AS price,
    pti.advance AS advance,
    pti.royalty AS royalty,
    psa.qty AS qty,
    pta.au_id AS au_id,
    pau.au_fname AS au_fname,
    pau.au_lname AS au_lname, 
    pta.royaltyper AS royaltyper,
    (pti.price * psa.qty * (pti.royalty / 100) * (pta.royaltyper / 100)) AS royalties

FROM 
    `ironhack-data-analytics-265219.publications.titleauthor` as pta

JOIN 
    publications.authors as pau
    
ON
    pta.au_id = pau.au_id 

JOIN
    publications.titles as pti

ON 
    pta.title_id = pti.title_id

JOIN
    publications.sales as psa

ON 
    pta.title_id = psa.title_id
    

GROUP BY 
    pti.title_id,
    pti.price,
    pti.advance,
    pti.royalty,
    psa.qty,
    pta.au_id,
    pau.au_lname,
    pau.au_fname,
    pta.royaltyper
    
ORDER BY 
    pti.title_id
    
LIMIT 5

'''

In [65]:
query_job = client.query(query=step1)

In [66]:
query_job.to_dataframe()

,title_id,price,advance,royalty,qty,au_id,au_fname,au_lname,royaltyper,royalties
0,BU1032,19.99,5000.0,10,5,409-56-7008,Abraham,Bennet,60,5.997
1,BU1032,19.99,5000.0,10,10,409-56-7008,Abraham,Bennet,60,11.994
2,BU1032,19.99,5000.0,10,5,213-46-8915,Marjorie,Green,40,3.998
3,BU1032,19.99,5000.0,10,10,213-46-8915,Marjorie,Green,40,7.996
4,BU1111,11.95,5000.0,10,25,724-80-9391,Stearns,MacFeather,60,17.925


### Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

- Title ID
- Author ID
- Aggregated royalties of each title for each author

Hint: use the SUM subquery and group by both au_id and title_id
In the output of this step, each title should appear only once for each author.

In [67]:
step2 = '''

SELECT
    step1.title_id,
    step1.au_id,
    step1.au_fname,
    step1.au_lname,
    step1.advance,
    SUM(royalties) as royalties

FROM 
    (SELECT
    pti.title_id AS title_id,
    pti.price AS price,
    pti.advance AS advance,
    pti.royalty AS royalty,
    psa.qty AS qty,
    pta.au_id AS au_id,
    pau.au_lname AS au_lname,
    pau.au_fname AS au_fname, 
    pta.royaltyper AS royaltyper,
    (pti.price * psa.qty * (pti.royalty / 100) * (pta.royaltyper / 100)) AS royalties

    FROM 
        `ironhack-data-analytics-265219.publications.titleauthor` as pta

    JOIN 
        publications.authors as pau
    
    ON
        pta.au_id = pau.au_id 

    JOIN
        publications.titles as pti

    ON 
        pta.title_id = pti.title_id

    JOIN
        publications.sales as psa

    ON 
        pta.title_id = psa.title_id
    

    GROUP BY 
        pti.title_id,
        pti.price,
        pti.advance,
        pti.royalty,
        psa.qty,
        pta.au_id,
        pau.au_lname,
        pau.au_fname,
        pta.royaltyper
    
    ORDER BY 
        pti.title_id) AS step1 
        
GROUP BY
    step1.title_id,
    step1.au_id,
    step1.au_lname,
    step1.au_fname,
    step1.advance
        
LIMIT 5

'''

In [68]:
query_job = client.query(query=step2)

In [69]:
query_job.to_dataframe()

,title_id,au_id,au_fname,au_lname,advance,royalties
0,TC3218,807-91-6654,Sylvia,Panteley,7000.0,83.800
1,MC3021,722-51-5454,Michel,DeFrance,15000.0,21.528
2,MC2222,712-45-1867,Innes,del Castillo,0.0,23.988
3,PS2091,899-46-2035,Anne,Ringer,2275.0,70.956
4,MC3021,899-46-2035,Anne,Ringer,15000.0,7.176


### Step 3: Calculate the total profits of each author
Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

- Author ID
- Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [73]:
step3 = '''

SELECT
    au_id,
    au_fname,
    au_lname,
    SUM(advance + royalties) AS total_profits
    
FROM
    (SELECT
        pti.title_id AS title_id,
        pti.price AS price,
        pti.advance AS advance,
        pti.royalty AS royalty,
        psa.qty AS qty,
        pta.au_id AS au_id,
        pau.au_lname AS au_lname,
        pau.au_fname AS au_fname, 
        pta.royaltyper AS royaltyper,
        (pti.price * psa.qty * (pti.royalty / 100) * (pta.royaltyper / 100)) AS royalties

    FROM 
        `ironhack-data-analytics-265219.publications.titleauthor` as pta

    JOIN 
        publications.authors as pau
    
    ON
        pta.au_id = pau.au_id 

    JOIN
        publications.titles as pti

    ON 
        pta.title_id = pti.title_id

    JOIN
        publications.sales as psa

    ON 
        pta.title_id = psa.title_id
    

    GROUP BY 
        pti.title_id,
        pti.price,
        pti.advance,
        pti.royalty,
        psa.qty,
        pta.au_id,
        pau.au_lname,
        pau.au_fname,
        pta.royaltyper) step1
        
GROUP BY 
    au_id,
    au_lname,
    au_fname

ORDER BY 
    total_profits DESC
    
LIMIT 3            

'''

In [74]:
query_job = client.query(query=step3)

In [75]:
query_job.to_dataframe()

,au_id,au_fname,au_lname,total_profits
0,899-46-2035,Anne,Ringer,39178.132
1,722-51-5454,Michel,DeFrance,30021.528
2,213-46-8915,Marjorie,Green,20162.110


### Challenge 2

Elevating from your solution in Challenge 1 , create a table named most_profiting_authors to hold the data about the most profiting authors. The table should have 2 columns:

- au_id - Author ID
- profits - The profits of the author aggregating the advances and royalties

Include your solution in solutions.ipynb.

In [ ]:
most_profiting_authors = '''

CREATE TABLE 
    `ironhack-data-analytics-265219.publications.most_profiting_authors` (
    
    au_id STRING,
    au_fname STRING,
    au_lname STRING,
    total_profits NUMERIC)

In [ ]:
sql= '''

INSERT INTO `ironhack-data-analytics-265219.publications.most_profiting_authors`

VALUES
('899-46-2035', 'Anne', 'Ringer', 39178.132),
('722-51-5454', 'Michel', 'DeFrance', 30021.528),
('213-46-8915', 'Marjorie','Green', 20162.110)
'''